In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool
from scipy.optimize import minimize
from datetime import datetime
import warnings
import joblib
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

In [2]:
# Create a log filename with the notebook name and current datetime
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f'kaggle_submission_{current_time}.log'

# Configure logging to save to a file
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()  # This ensures logs are also output to the console
    ]
)

In [3]:
def reduce_mem_usage(df, verbose=True):
    """Iterate through all the columns of a dataframe and modify the data type to reduce memory usage."""
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        logging.info(f'Start memory usage of dataframe: {start_mem:.2f} MB')


    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        logging.info(f'End memory usage of dataframe: {end_mem:.2f} MB')
        logging.info(f'Decreased by {(100 * (start_mem - end_mem) / start_mem):.1f}%')

    return df

def safe_map(df, column, mapping):
    """Map categorical values to numerical values and log any unknown categories."""
    unknown_categories = set(df[column]) - set(mapping.keys())
    if unknown_categories:
        logging.warning(f"Unknown categories in column {column}: {unknown_categories}")
    df[column] = df[column].map(mapping)
    return df

def import_data(file, **kwargs):
    """Create a dataframe and optimize its memory usage."""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True, **kwargs)
    df = reduce_mem_usage(df)
    return df

def preprocess_data(df):
    """Preprocess the dataset."""
    gender_mapping = {'Male': 1, 'Female': 0}
    vehicle_damage_mapping = {'Yes': 1, 'No': 0}
    vehicle_age_mapping = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}
    
    df = safe_map(df, 'Gender', gender_mapping)
    df = safe_map(df, 'Vehicle_Damage', vehicle_damage_mapping)
    df = safe_map(df, 'Vehicle_Age', vehicle_age_mapping)
    
    df.drop(['Driving_License'], axis=1, inplace=True)
    return df

def feature_engineering(df):
    """Feature engineering on the dataset."""
    df['Previously_Insured_Annual_Premium'] = pd.factorize((df['Previously_Insured'].astype(str) + df['Annual_Premium'].astype(str)))[0]
    df['Previously_Insured_Vehicle_Age'] = pd.factorize((df['Previously_Insured'].astype(str) + df['Vehicle_Age'].astype(str)))[0]
    df['Previously_Insured_Vehicle_Damage'] = pd.factorize((df['Previously_Insured'].astype(str) + df['Vehicle_Damage'].astype(str)))[0]
    df['Previously_Insured_Vintage'] = pd.factorize((df['Previously_Insured'].astype(str) + df['Vintage'].astype(str)))[0]
    return df



In [4]:
# Paths to datasets
train_path = r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\train.csv"
test_path = r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\test.csv"

# Load and optimize data
train_df = import_data(train_path, index_col='id')
test_df = import_data(test_path, index_col='id')

gc.collect()
print(f"DataFrame after import: {type(train_df)}")
logging.info("Data loaded successfully.")

# Apply preprocessing
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)
print(f"DataFrame after preprocessing: {type(train_df)}")
logging.info("Data preprocessed successfully.")

# Apply feature engineering
train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)

gc.collect()
print(f"DataFrame after feature engineering: {type(train_df)}")
logging.info("Feature engineering completed successfully.")

2024-07-25 15:47:53,040 - INFO - Start memory usage of dataframe: 1053.30 MB
2024-07-25 15:47:54,721 - INFO - End memory usage of dataframe: 318.18 MB
2024-07-25 15:47:54,722 - INFO - Decreased by 69.8%
2024-07-25 15:48:04,902 - INFO - Start memory usage of dataframe: 643.68 MB
2024-07-25 15:48:06,038 - INFO - End memory usage of dataframe: 204.81 MB
2024-07-25 15:48:06,040 - INFO - Decreased by 68.2%
2024-07-25 15:48:06,151 - INFO - Data loaded successfully.


DataFrame after import: <class 'pandas.core.frame.DataFrame'>


2024-07-25 15:48:08,106 - INFO - Data preprocessed successfully.


DataFrame after preprocessing: <class 'pandas.core.frame.DataFrame'>


2024-07-25 15:48:41,630 - INFO - Feature engineering completed successfully.


DataFrame after feature engineering: <class 'pandas.core.frame.DataFrame'>


In [5]:
num_cols = ['Age', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
scaler = StandardScaler()
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

# Separate features and target variable
X = train_df.drop('Response', axis=1)
y = train_df['Response']

In [6]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import joblib
import numpy as np
import gc
import logging

# Define the objective function for the Optuna study
def objective(trial):
    # Define parameter search space with realistic ranges
    lgb_params = {
        'objective': 'binary',
        'metric': 'auc',
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-2, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-2, 10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.1),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'num_leaves': trial.suggest_int('num_leaves', 20, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-3, 10.0),
        'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-3, 1.0),
        'max_bin': trial.suggest_int('max_bin', 100, 300),
        'device': 'gpu',
        'early_stopping_round': 50
    }

    # Suggest number of boosting rounds
    num_boost_round = trial.suggest_int('num_boost_round', 2500, 5000)

    # Split the data into training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

    # Train the model
    model = lgb.train(
        lgb_params,
        train_data,
        num_boost_round=num_boost_round,
        valid_sets=[train_data, valid_data],
    )

    # Predict and calculate AUC score
    valid_preds = model.predict(X_valid, num_iteration=model.best_iteration)
    auc_score = roc_auc_score(y_valid, valid_preds)

    # Save the model from this trial
    joblib.dump(model, f'model_trial_{trial.number}.pkl')

    return auc_score

# Create and optimize the study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best parameters and AUC score
best_params = study.best_params
best_score = study.best_value

logging.info(f"Best AUC score: {best_score:.6f}")
logging.info(f"Best parameters: {best_params}")

# Load the best model
best_model = joblib.load(f'model_trial_{study.best_trial.number}.pkl')

# Predict on the test set
test_pred = best_model.predict(test_df, num_iteration=best_model.best_iteration)
joblib.dump(test_pred, 'test_pred_lgb.pkl')

logging.info("Final LightGBM model and predictions saved.")


[I 2024-07-25 15:48:43,261] A new study created in memory with name: no-name-c5cbef46-8686-4075-a95b-057a45e5fd34


[LightGBM] [Info] Number of positive: 1132047, number of negative: 8071791
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 929
[LightGBM] [Info] Number of data points in the train set: 9203838, number of used features: 13
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2070 SUPER, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (140.44 MB) transferred to GPU in 0.131377 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122997 -> initscore=-1.964348
[LightGBM] [Info] Start training from score -1.964348
Training until validation scores don't improve for 50 rounds


In [ ]:
# Create a submission dataframe
submission = pd.DataFrame({
    'id': test_df.index,
    'Response': test_pred
})

# Save the submission file
submission_filename = f'submission_{current_time}.csv'
submission.to_csv(submission_filename, index=False)

logging.info(f"Submission file {submission_filename} created successfully.")
print(f"Submission file {submission_filename} created successfully.")

In [ ]:
optuna.visualization.plot_optimization_history(study)


In [ ]:

optuna.visualization.plot_intermediate_values(study)


In [ ]:

optuna.visualization.plot_parallel_coordinate(study)


In [ ]:

optuna.visualization.plot_contour(study)


In [ ]:

optuna.visualization.plot_param_importances(study)


In [ ]:

optuna.visualization.plot_slice(study)


In [ ]:

optuna.visualization.plot_edf(study)